# ProHMR Demo Notebook
This a demo notebook for our paper _Probabilistic Modeling for Human Mesh Recovery_.

Project webpage: https://www.seas.upenn.edu/~nkolot/projects/prohmr

Github repo: https://github.com/nkolot/ProHMR

Instructions:

1. Enable the GPU Runtime (Runtime > Change Runtime Type > GPU)
2. Install OpenPose (~15 minutes)
3. Download and install depencencies
4. Fetch data
6. Run the demo

## Acknowledgements
- The OpenPose installation code is taken from [this notebook](https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb).
- The structure of this notebook follows closely [PHOSA](https://colab.research.google.com/drive/1QIoL2g0jdt5E-vYKCIojkIz21j3jyEvo?usp=sharing).



# Install OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

# Download and install dependencies

First we will clone the project repo and then install all necessary dependencies.

In [ ]:
!git clone https://github.com/nkolot/ProHMR.git
!pip install -r ProHMR/colab_requirements.txt
!cd ProHMR && pip install .

Cloning into 'ProHMR'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 97 (delta 18), reused 71 (delta 5), pack-reused 0
Unpacking objects: 100% (97/97), done.
  Cloning https://github.com/nkolot/nflows.git to /tmp/pip-req-build-3zkr535f
  Running command git clone -q https://github.com/nkolot/nflows.git /tmp/pip-req-build-3zkr535f
     |████████████████████████████████| 1.1 MB 12.9 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 241 kB 55.4 MB/s 
     |████████████████████████████████| 379 kB 52.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 14.0 MB 20.6 MB/s 
     |████████████████████████████████| 112 kB 58.4 MB/s 
     |████████████████████████████████| 3.7 MB 45.5 MB/s 
     |████████████████████████████████| 414 kB 47.0 MB/s 
     |████████████████████████████████| 1.2 M

Processing /content/ProHMR
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for prohmr: filename=prohmr-0.0.0-py3-none-any.whl size=51004 sha256=3a192e6c2227c4d74ba46541192e864b2f5717b479310dd3aea9f744692863b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-71cw7u2i/wheels/2e/d5/8f/f1016c1be4ab392abe4f8164ce31362549de4d14033ddef506
Successfully built prohmr


# Fetch model data

In [ ]:
!cd ProHMR && bash fetch_data.sh

--2022-01-05 02:59:38--  http://visiondata.cis.upenn.edu/prohmr/data.tar.gz
Resolving visiondata.cis.upenn.edu (visiondata.cis.upenn.edu)... 158.130.68.16
Connecting to visiondata.cis.upenn.edu (visiondata.cis.upenn.edu)|158.130.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 641669120 (612M) [application/x-gzip]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>] 611.94M  36.4MB/s    in 17s     

2022-01-05 02:59:55 (35.4 MB/s) - ‘data.tar.gz’ saved [641669120/641669120]

data/
data/SMPL_to_J19.pkl
data/smpl_mean_params.npz
data/checkpoint.pt
--2022-01-05 02:59:58--  http://visiondata.cis.upenn.edu/prohmr/datasets.tar.gz
Resolving visiondata.cis.upenn.edu (visiondata.cis.upenn.edu)... 158.130.68.16
Connecting to visiondata.cis.upenn.edu (visiondata.cis.upenn.edu)|158.130.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251111104 (239M) [application/x-gzip]
Saving to: ‘datasets.tar.gz’

datasets.tar.gz   

# Fetch SMPL model

In [ ]:
# We are downloading the SMPL model here for convenience. Please accept the license
# agreement on the SMPL website: https://smpl.is.tue.mpg.
!mkdir -p ProHMR/data/smpl
!wget https://github.com/classner/up/raw/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl

--2022-01-05 03:00:21--  https://github.com/classner/up/raw/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/classner/up/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl [following]
--2022-01-05 03:00:21--  https://raw.githubusercontent.com/classner/up/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39001280 (37M) [application/octet-stream]
Saving to: ‘basicModel_neutral_lbs_10_207_0_v1.0.0.pkl’

basicModel_neutral_ 100%[===================>]  37.19M   247MB/s    in 0

In [ ]:
# Convert SMPL pkl file to be compatible with Python 3
# Script is from https://rebeccabilbro.github.io/convert-py2-pickles-to-py3/
import os
import dill
import pickle
import argparse

def convert(old_pkl):
    """
    Convert a Python 2 pickle to Python 3
    """
    # Make a name for the new pickle
    new_pkl = os.path.splitext(os.path.basename(old_pkl))[0]+"_p3.pkl"

    # Convert Python 2 "ObjectType" to Python 3 object
    dill._dill._reverse_typemap["ObjectType"] = object

    # Open the pickle using latin1 encoding
    with open(old_pkl, "rb") as f:
        loaded = pickle.load(f, encoding="latin1")

    # Re-save as Python 3 pickle
    with open(new_pkl, "wb") as outfile:
        pickle.dump(loaded, outfile)

convert('basicModel_neutral_lbs_10_207_0_v1.0.0.pkl')
!rm basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
!mv basicModel_neutral_lbs_10_207_0_v1.0.0_p3.pkl ProHMR/data/smpl/SMPL_NEUTRAL.pkl

# Reconstruct Poses from a YouTube Video

In [ ]:
from IPython.display import YouTubeVideo
YOUTUBE_ID = 'UWawJcxYY6I'


YouTubeVideo(YOUTUBE_ID)

In [ ]:
# download the youtube with the given ID
!pip install youtube-dl
!rm youtube.mp4
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
!rm -r images
!mkdir -p images
# extract a 15 second part of the video to an .mp4
!ffmpeg -y -ss 00:00:40.00 -i youtube.mp4 -t 15 "images/frame_%06d.jpg"
# Get video fps
framerate = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate youtube.mp4
framerate = str(framerate[0])
# detect poses on the these 5 seconds and save the json files
!cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../images/ --write_json ../keypoints --display 0 --render_pose 0

In [ ]:
# Run ProHMR + fitting
# We use the demo code provided by the repo
!cd ProHMR && python demo.py --img_folder=../images --keypoint_folder=../keypoints --out_folder=../output --run_fitting --full_frame --batch_size=64

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100% 97.8M/97.8M [00:01<00:00, 86.2MB/s]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/saving.py:209: UserWarning: Found keys that are in the model state dict but not in the checkpoint: ['smpl.shapedirs', 'smpl.faces_tensor', 'smpl.v_template', 'smpl.J_regressor', 'smpl.posedirs', 'smpl.parents', 'smpl.lbs_weights', 'smpl.joint_regressor_extra', 'smpl.joint_map', 'smpl.vertex_joint_selector.extra_joints_idxs']
  f"Found keys that are in the model state dict but not in the checkpoint: {keys.missing_keys}"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/saving.py:213: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['flow.flow._distribution.dummy_buffer']
  f"Found keys that are not in the model state dict but in the checkpoint: {keys.unexpected_keys}"
100% 1/1 [00:08<00:00,  8.29s/it]


In [ ]:
# Create video from reconstructed frames
!ffmpeg -r $framerate -i output/frame_%06d_fitting.jpg -vcodec libx264 -crf 25  -pix_fmt yuv420p fitting_output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# Display the reconstruction video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('fitting_output.mp4', width=960, height=540)